# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#getting data: the dataset was uploaded to and is accesible through my github repo
from azureml.data.dataset_factory import TabularDatasetFactory

rawdata_homic2018 = "https://raw.githubusercontent.com/hftamayo/azuremlproj03/main/cad2018.csv"
dshomic2018 = TabularDatasetFactory.from_delimited_files(path=rawdata_homic2018, separator=',')
hyperpdhomic2018 = dshomic2018.to_pandas_dataframe().dropna()

In [3]:
ws = Workspace.from_config()
expname = 'hyperp3ht'

exphyper=Experiment(ws, expname)

In [4]:
# Compute name has restrictions, please refer to official documentation
ccname = "ccp03hyp"
try:
    ccluster = ComputeTarget(ws, ccname)
    print(f"{ccname} exists already")
except:
    ccconfig = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=5)
    ccluster = ComputeTarget.create(ws, ccname, ccconfig)
ccluster.wait_for_completion(show_output=True)


Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [17]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
#param_sampling = RandomParameterSampling({"--C": choice(1,2,3,4,5), "--max_iter": choice(80,100,120,150,170,200)})
param_sampling = RandomParameterSampling({"--C": choice(1,2,3,4), "--max_iter": choice(40,80,120,130,200)})

if "outputs" not in os.listdir():
    os.mkdir("./outputs")
    
#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='.', compute_target=ccluster, entry_script='train.py')

hyperdrive_run_config = HyperDriveConfig(estimator=estimator, policy=early_termination_policy, primary_metric_name="Accuracy",
                                          hyperparameter_sampling=param_sampling,
                                         max_total_runs=20,
                                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)

In [18]:
#TODO: Submit your experiment
hyperDrive_run = exphyper.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [19]:
RunDetails(hyperDrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [20]:
brmodel = hyperDrive_run.get_best_run_by_primary_metric()
brmetrics = brmodel.get_metrics()

print('Best Run Id: ', brmodel.id)
print('\n Accuracy:', brmetrics['Accuracy'])
print('\n Regularization Strength:',brmetrics['Regularization Strength:'])
print('\n Max Iterations:',brmetrics['Max iterations:'])

Best Run Id:  HD_6a6c0070-9185-420b-af26-284a46fb5ae7_6

 Accuracy: 0.8914342629482072

 Regularization Strength: 1.0

 Max Iterations: 120


In [21]:
#this lines is really useful in case you want to check files related to the model
brmodel.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_c6b08b7f163ecec0ba509382f1ba598cf4f7cfef64ad6d43f1da46e62e320312_d.txt',
 'azureml-logs/65_job_prep-tvmps_c6b08b7f163ecec0ba509382f1ba598cf4f7cfef64ad6d43f1da46e62e320312_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_c6b08b7f163ecec0ba509382f1ba598cf4f7cfef64ad6d43f1da46e62e320312_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/105_azureml.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/bhypermodel.pkl']

In [22]:
#TODO: Save the best model: the files is created and uploaded during the execution of train.py
model = brmodel.register_model(model_name = 'bhypermodel', model_path = 'outputs/bhypermodel.pkl') 
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-136496', subscription_id='1b944a9b-fdae-4f97-aeb1-b7eea0beac53', resource_group='aml-quickstarts-136496'), name=bhypermodel, id=bhypermodel:2, version=2, tags={}, properties={})


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [23]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
env=Environment.from_conda_specification(name="hyperenv",
                                                  file_path="hyperenv.yml")

#myenv=Environment.from_conda_specification(name="myenv",
#                                                  file_path="./conda_dephyper.yml")
#env = Environment.get(workspace=ws, name="AzureML-AutoML")
#OPCIONAL: env = Environment.get(workspace, "AzureML-Minimal").clone(env_name)

In [24]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig, Model

infcfg = InferenceConfig(entry_script='score.py',
                                    environment=env)

# deploying the model via WebService
from azureml.core.webservice import AciWebservice

depcfg = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
webservice = Model.deploy(ws, "wshypp03ht01", [model], infcfg, depcfg)
webservice.wait_for_deployment(show_output = True)
print(webservice.state)

print(webservice.scoring_uri)

print(webservice.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.............................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://4115e61b-0d9d-4629-8598-c56894967dbd.southcentralus.azurecontainer.io/score
http://4115e61b-0d9d-4629-8598-c56894967dbd.southcentralus.azurecontainer.io/swagger.json


In [25]:
import json

#Use a test case in particular from the dataset
#data = {"data":
#        [
#          {
#           "id": 2125, "regfalle": 21917, "edad": 42, "deptoocuhe": 2, "tipoarma": 1, "pracaut": 1
#          },
#      ]
#    }
data = [[2125, 21917, 42, 2, 1, 1]]
         
# Convert to JSON string
tstdatahomic2018 = json.dumps(data)

print(tstdatahomic2018)

[[2125, 21917, 42, 2, 1, 1]]


TODO: In the cell below, send a request to the web service you deployed to test it.

In [26]:
import requests

headers = {'Content-type': 'application/json'}

response = requests.post(webservice.scoring_uri, tstdatahomic2018, headers=headers)

In [30]:
# Print results
print(response.status_code)
print(response.text)

200
[0]


TODO: In the cell below, print the logs of the web service and delete the service

In [28]:
print(webservice.get_logs())

2021-01-29T12:09:16,601308249+00:00 - rsyslog/run 
2021-01-29T12:09:16,601308349+00:00 - iot-server/run 
2021-01-29T12:09:16,603478376+00:00 - gunicorn/run 
2021-01-29T12:09:16,605247297+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libuuid.so.1: no version information available (required by rsyslogd)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml

In [31]:
#BEST PRACTICES: deleting webservice and CCluster that won't be used anymore
webservice.delete()
ccluster.delete()